# This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
    Note: currently this requires pynwb 1.3.3 or above and hdmf 2.0.1 or above, so use the latest development versions, not the conda versions
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

#### Load all of the relevant modules and set the environment variables. 
Note that the datadir and datadir/analysis must exist

In [13]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

# the commands below can be run once to update your global configuration

dj.config["enable_python_native_blobs"] = True
dj.config["database.user"] = 'root'
dj.config["databasse.password"] = 'tutorial'
dj.config.save_global()

import nwb_datajoint as nd
import ndx_franklab_novela.probe

data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['NWB_DATAJOINT_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')



env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [14]:
nwb_file_names = nd.common.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

beans20190718.nwb


#### Create the standard LFP Filters. This only needs to be done once.

In [15]:
nd.common.FirFilter().create_standard_filters()

### Select all electrodes for LFP

In [16]:
electrode_ids = nd.common.Electrode.fetch('electrode_id')
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, electrode_ids.tolist())


About to delete:
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 256 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  yes


Committed.


### Or select one electrode for LFP


In [10]:
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, [2])


About to delete:
`common_ephys`.`__l_f_p_band`: 1 items
`common_ephys`.`l_f_p_band_selection__l_f_p_band_electrode`: 3 items
`common_ephys`.`l_f_p_band_selection`: 1 items
`common_ephys`.`_l_f_p`: 1 items
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 256 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  yes


Committed.


### Populate the LFP table. Note that this takes an hour or so on a laptop if you use all electrodes

In [ ]:
import time
tic = time.perf_counter()
nd.common.LFP().populate()
toc = time.perf_counter()
toc - tic

writing new NWB file beans20190718_00000000.nwb
Output array should have shape (1091950, 256) and dtype <f8
Output array should have shape (212812, 256) and dtype <f8
Output array should have shape (2557744, 256) and dtype <f8
Output array should have shape (1356939, 256) and dtype <f8
Output array should have shape (1680945, 256) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 256)
Checking output array shape is disabled, make sure portion of output array has shape (212812, 256)
Checking output array shape is disabled, make sure portion of output array has shape (2557744, 256)


### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [8]:
lfp_sampling_rate = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch1('lfp_sampling_rate')
filter_name = 'Theta 5-11 Hz'
nd.common.FirFilter().add_filter(filter_name, lfp_sampling_rate, 'bandpass', [4, 5, 11, 12], 'theta filter for 1 Khz data')

Next we add an entry for the LFP Band and the electrodes we want to filter

In [6]:
# assume that we've filtered all the electrodes; change this if not
lfp_band_electrode_ids = [1, 3, 5]

# set the interval list name corresponding to the second epoch (a run session)
interval_list_name = '02_r1'

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = lfp_sampling_rate // 10

[autoreload of nwb_datajoint.common.common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packag

NameError: name 'lfp_sampling_rate' is not defined

In [9]:
nd.common.LFPBandSelection().set_lfp_band_electrodes(nwb_file_name, lfp_band_electrode_ids, filter_name, interval_list_name, ref_elect, lfp_band_sampling_rate)

[autoreload of nwb_datajoint.common.common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packag

DuplicateError: ("Duplicate entry 'beans20190718.nwb-Theta 5-11 Hz-1000-0-1--1' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

Check to make sure it worked

In [4]:
nd.common.LFPBandSelection().LFPBandElectrode()

nwb_file_name the name of the NWB file,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,reference_elect_id the reference electrode to use; -1 for no reference
beans20190718.nwb,Theta 5-11 Hz,1000,0,1,-1
beans20190718.nwb,Theta 5-11 Hz,1000,0,3,-1
beans20190718.nwb,Theta 5-11 Hz,1000,0,5,-1


In [3]:
nd.common.LFPBand().populate()

Theta 5-11 Hz 100
writing new NWB file beans20190718_00000001.nwb
Output array should have shape (277056, 3) and dtype <f8
[277056, 3]
1091950 3862506 10
Diffs [0.00999975 0.00999999 0.00999999 ... 0.00999999 0.00999999 0.00999999]
Checking output array shape is disabled, make sure portion of output array has shape (277056, 3)


In [5]:
nd.common.LFPBand()

nwb_file_name the name of the NWB file,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name the name of the file,nwb_object_id the NWB object ID for loading this object from the file
beans20190718.nwb,Theta 5-11 Hz,1000,beans20190718_00000001.nwb,878e7d87-1f29-4af4-85b5-1d5e259c4b26


In [18]:
nd.common.AnalysisNwbfile.delete()

About to delete:
`common_lab`.`analysis_nwbfile`: 2 items


Proceed? [yes, No]:  yes


Committed.


In [19]:
import numpy as np
b = np.asarray(n.data, dtype=type(n.data[0][0]))

In [20]:
b

array([[-165, -140,  -74, ..., -332, -238, -262],
       [ -76,  -42, -128, ..., -669, -396, -403],
       [  79,   38, -120, ..., -544, -207, -196],
       ...,
       [  -8,  -40,  139, ...,   83,  232,  274],
       [ 264,  224,  181, ...,  149,  303,  382],
       [ 320,  312,  158, ...,  144,  260,  284]], dtype=int16)